
# 0) Evaluating RAG Retrievers (Hugging Face) — Extended
**Audience:** 4th‑year CS students  
**Goal:** Quantify retrieval performance for a RAG pipeline using IR metrics, and compare **dense**, **BM25**, **hybrid**, and **reranked** approaches. Optionally, run an **end‑to‑end** generation step with a small model.

**What you'll build and measure:**
- Core IR metrics: **Precision@5/@10**, **Recall@5/@10**, **MAP**, **MRR**
- Dense retriever baseline with `sentence-transformers/all-MiniLM-L6-v2`
- **(6)** BM25 sparse baseline (`rank_bm25`)
- **(7)** Hybrid dense + sparse (score fusion)
- **(9)** Cross‑encoder reranker (`cross-encoder/ms-marco-MiniLM-L-6-v2`)
- **(11)** Optional end‑to‑end generation using `google/flan-t5-base` (Apache‑2.0)

### Dataset (Legal)
We use **SQuAD v1.1** (via 🤗 `datasets`) — **CC BY‑SA 4.0** — a widely used QA dataset.  
- **Document** = unique `context` paragraph  
- **Query** = the corresponding `question`  
- **Ground truth** = the `doc_id` of the paragraph that generated the question

> For clarity and speed, we assume **one relevant document per query**.


In [ ]:

# ======= 0.1) Install dependencies (uncomment if needed) =======
# !pip install -U datasets transformers sentence-transformers rank_bm25 numpy pandas tqdm matplotlib
# Optional for scale:
# !pip install faiss-cpu


In [ ]:

# ======= 0.2) Imports and configuration =======
from datasets import load_dataset
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from transformers import pipeline

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

N_EXAMPLES = 400
MAX_DOCS = 350
TOP_KS = [5, 10]

DENSE_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
RERANKER_MODEL_NAME = "cross-encoder/ms-marco-MiniLM-L-6-v2"
GEN_MODEL_NAME = "google/flan-t5-base"


## 1) Load dataset and construct corpus + queries

In [ ]:

# ======= 1.1) Load SQuAD v1.1 =======
squad = load_dataset("squad", split="validation")
df = squad.to_pandas()[["id", "title", "context", "question", "answers"]]
df = df.sample(frac=1.0, random_state=RANDOM_SEED).head(N_EXAMPLES).reset_index(drop=True)
print(f"Loaded {len(df)} SQuAD examples.")
df.head(3)


In [ ]:

# ======= 1.2) Build corpus of unique documents (contexts) =======
unique_contexts = pd.Series(df["context"].unique())
if len(unique_contexts) > MAX_DOCS:
    unique_contexts = unique_contexts.sample(n=MAX_DOCS, random_state=RANDOM_SEED).reset_index(drop=True)

doc_df = pd.DataFrame({
    "doc_id": np.arange(len(unique_contexts), dtype=int),
    "text": unique_contexts
})
context_to_docid = {ctx: i for i, ctx in enumerate(unique_contexts)}

print(f"Corpus size (unique documents): {len(doc_df)}")
doc_df.head(2)


In [ ]:

# ======= 1.3) Build query set with ground-truth doc IDs =======
def map_context_to_docid(ctx):
    return context_to_docid.get(ctx, None)

query_rows = []
for _, row in df.iterrows():
    did = map_context_to_docid(row["context"])
    if did is None:
        continue
    query_rows.append({
        "question_id": row["id"],
        "question": row["question"],
        "relevant_doc_id": int(did),
        "title": row["title"]
    })
query_df = pd.DataFrame(query_rows).reset_index(drop=True)
print(f"Queries retained after trimming: {len(query_df)}")
query_df.head(3)


## 2) Dense embeddings (bi‑encoder)

In [ ]:

# ======= 2.1) Encode documents and queries =======
dense_model = SentenceTransformer(DENSE_MODEL_NAME)
doc_embeddings = dense_model.encode(doc_df["text"].tolist(), convert_to_numpy=True, show_progress_bar=True)
query_embeddings = dense_model.encode(query_df["question"].tolist(), convert_to_numpy=True, show_progress_bar=True)
doc_embeddings.shape, query_embeddings.shape


In [ ]:

# ======= 2.2) Dense retrieval helper (cosine similarity) =======
def dense_ranked_docs_for_query(q_vec, doc_vecs):
    q = q_vec / (np.linalg.norm(q_vec) + 1e-12)
    D = doc_vecs / (np.linalg.norm(doc_vecs, axis=1, keepdims=True) + 1e-12)
    sims = D @ q
    ranked_doc_ids = np.argsort(-sims)
    return ranked_doc_ids, sims[ranked_doc_ids]


## 3) Metrics

In [ ]:

# ======= 3.1) Metric implementations =======
def precision_at_k(ranked_doc_ids, relevant_doc_ids, k):
    ranked_k = ranked_doc_ids[:k]
    hits = sum(1 for d in ranked_k if d in relevant_doc_ids)
    return hits / float(k)

def recall_at_k(ranked_doc_ids, relevant_doc_ids, k):
    ranked_k = ranked_doc_ids[:k]
    hits = sum(1 for d in ranked_k if d in relevant_doc_ids)
    return hits / float(len(relevant_doc_ids)) if len(relevant_doc_ids) > 0 else 0.0

def average_precision(ranked_doc_ids, relevant_doc_ids):
    if len(relevant_doc_ids) == 0:
        return 0.0
    hits = 0
    precisions = []
    for i, d in enumerate(ranked_doc_ids, start=1):
        if d in relevant_doc_ids:
            hits += 1
            precisions.append(hits / i)
    return float(np.mean(precisions)) if precisions else 0.0

def reciprocal_rank(ranked_doc_ids, relevant_doc_ids):
    for i, d in enumerate(ranked_doc_ids, start=1):
        if d in relevant_doc_ids:
            return 1.0 / i
    return 0.0

def evaluate_ranked_list(ranked_doc_ids, relevant_doc_id):
    relevant = {int(relevant_doc_id)}
    metrics = {
        "P@5": precision_at_k(ranked_doc_ids, relevant, 5),
        "P@10": precision_at_k(ranked_doc_ids, relevant, 10),
        "R@5": recall_at_k(ranked_doc_ids, relevant, 5),
        "R@10": recall_at_k(ranked_doc_ids, relevant, 10),
        "AP": average_precision(ranked_doc_ids, relevant),
        "RR": reciprocal_rank(ranked_doc_ids, relevant),
    }
    pos = np.where(ranked_doc_ids == list(relevant)[0])[0]
    metrics["rank_of_relevant"] = int(pos[0]) + 1 if len(pos) else np.inf
    return metrics


## 4) Dense baseline — evaluation

In [ ]:

dense_records = []
for i in tqdm(range(len(query_df)), desc="Dense baseline: evaluating queries"):
    ranked_docs, _ = dense_ranked_docs_for_query(query_embeddings[i], doc_embeddings)
    m = evaluate_ranked_list(ranked_docs, query_df.iloc[i]["relevant_doc_id"])
    dense_records.append(m)

dense_df = pd.DataFrame(dense_records)
dense_summary = pd.DataFrame({
    "Metric": ["MAP", "MRR", "Precision@5", "Precision@10", "Recall@5", "Recall@10"],
    "Score":  [dense_df["AP"].mean(), dense_df["RR"].mean(),
               dense_df["P@5"].mean(), dense_df["P@10"].mean(),
               dense_df["R@5"].mean(), dense_df["R@10"].mean()]
})
dense_summary


## 5) Dense baseline — rank histogram

In [ ]:

plt.figure()
plt.hist(dense_df["rank_of_relevant"].replace(np.inf, np.nan).dropna(), bins=30)
plt.title("Dense Baseline — Rank of Relevant Document")
plt.xlabel("Rank (1 = top)")
plt.ylabel("Count")
plt.show()


## 6) BM25 sparse baseline

In [ ]:

# ======= 6.1) BM25 index =======
def simple_tokenize(text): 
    return text.lower().split()

bm25_corpus_tokens = [simple_tokenize(t) for t in doc_df["text"].tolist()]
bm25 = BM25Okapi(bm25_corpus_tokens)

def bm25_ranked_docs_for_query(query_text):
    q_tokens = simple_tokenize(query_text)
    scores = bm25.get_scores(q_tokens)
    ranked = np.argsort(-scores)
    return ranked, scores[ranked]


In [ ]:

bm25_records = []
for i in tqdm(range(len(query_df)), desc="BM25 baseline: evaluating queries"):
    ranked_docs, _ = bm25_ranked_docs_for_query(query_df.iloc[i]["question"])
    m = evaluate_ranked_list(ranked_docs, query_df.iloc[i]["relevant_doc_id"])
    bm25_records.append(m)

bm25_df = pd.DataFrame(bm25_records)
bm25_summary = pd.DataFrame({
    "Metric": ["MAP", "MRR", "Precision@5", "Precision@10", "Recall@5", "Recall@10"],
    "Score":  [bm25_df["AP"].mean(), bm25_df["RR"].mean(),
               bm25_df["P@5"].mean(), bm25_df["P@10"].mean(),
               bm25_df["R@5"].mean(), bm25_df["R@10"].mean()]
})
bm25_summary


## 7) Hybrid dense + BM25 (score fusion)

In [ ]:

def minmax_norm(x):
    x = np.asarray(x, dtype=float)
    mn, mx = np.min(x), np.max(x)
    if mx - mn < 1e-12:
        return np.zeros_like(x)
    return (x - mn) / (mx - mn)

def hybrid_ranked_docs_for_query(q_text, q_vec, doc_vecs, alpha=0.5):
    # Dense scores
    q = q_vec / (np.linalg.norm(q_vec) + 1e-12)
    D = doc_vecs / (np.linalg.norm(doc_vecs, axis=1, keepdims=True) + 1e-12)
    dense_scores = D @ q

    # BM25 scores
    q_tokens = q_text.lower().split()
    bm25_scores = bm25.get_scores(q_tokens)

    # Normalize and fuse
    dense_n = minmax_norm(dense_scores)
    bm25_n = minmax_norm(bm25_scores)
    fused = alpha * dense_n + (1 - alpha) * bm25_n

    ranked = np.argsort(-fused)
    return ranked, fused[ranked]


In [ ]:

alpha = 0.5
hybrid_records = []
for i in tqdm(range(len(query_df)), desc="Hybrid (dense+BM25): evaluating queries"):
    ranked_docs, _ = hybrid_ranked_docs_for_query(query_df.iloc[i]["question"], query_embeddings[i], doc_embeddings, alpha=alpha)
    m = evaluate_ranked_list(ranked_docs, query_df.iloc[i]["relevant_doc_id"])
    hybrid_records.append(m)

hybrid_df = pd.DataFrame(hybrid_records)
hybrid_summary = pd.DataFrame({
    "Metric": ["MAP", "MRR", "Precision@5", "Precision@10", "Recall@5", "Recall@10"],
    "Score":  [hybrid_df["AP"].mean(), hybrid_df["RR"].mean(),
               hybrid_df["P@5"].mean(), hybrid_df["P@10"].mean(),
               hybrid_df["R@5"].mean(), hybrid_df["R@10"].mean()]
})
hybrid_summary


## 8) Compare Dense vs BM25 vs Hybrid

In [ ]:

comparison = pd.DataFrame({
    "Metric": ["MAP", "MRR", "Precision@5", "Precision@10", "Recall@5", "Recall@10"],
    "Dense":  [dense_df["AP"].mean(), dense_df["RR"].mean(),
               dense_df["P@5"].mean(), dense_df["P@10"].mean(),
               dense_df["R@5"].mean(), dense_df["R@10"].mean()],
    "BM25":   [bm25_df["AP"].mean(), bm25_df["RR"].mean(),
               bm25_df["P@5"].mean(), bm25_df["P@10"].mean(),
               bm25_df["R@5"].mean(), bm25_df["R@10"].mean()],
    "Hybrid": [hybrid_df["AP"].mean(), hybrid_df["RR"].mean(),
               hybrid_df["P@5"].mean(), hybrid_df["P@10"].mean(),
               hybrid_df["R@5"].mean(), hybrid_df["R@10"].mean()],
})
comparison


## 9) Cross‑encoder reranker (re‑score top‑N)

In [ ]:

reranker = CrossEncoder(RERANKER_MODEL_NAME)  # downloads the model first time
TOP_N = 50
BASE_FOR_RERANK = "dense"  # choose "dense" or "hybrid"


In [ ]:

def build_full_reranked_list(q_text, base_ranked_ids, top_n=50):
    candidates = base_ranked_ids[:top_n].tolist()
    pairs = [(q_text, doc_df.loc[d, "text"]) for d in candidates]
    scores = reranker.predict(pairs)
    order = np.argsort(-np.array(scores))
    topN_reranked = [candidates[i] for i in order]
    remainder = [d for d in base_ranked_ids.tolist() if d not in set(candidates)]
    return np.array(topN_reranked + remainder, dtype=int)


In [ ]:

rerank_records = []
for i in tqdm(range(len(query_df)), desc="Reranker: evaluating queries"):
    q_text = query_df.iloc[i]["question"]
    if BASE_FOR_RERANK == "hybrid":
        base_ranked, _ = hybrid_ranked_docs_for_query(q_text, query_embeddings[i], doc_embeddings, alpha=0.5)
    else:
        base_ranked, _ = dense_ranked_docs_for_query(query_embeddings[i], doc_embeddings)

    full_reranked = build_full_reranked_list(q_text, base_ranked, top_n=TOP_N)
    m = evaluate_ranked_list(full_reranked, query_df.iloc[i]["relevant_doc_id"])
    rerank_records.append(m)

rerank_df = pd.DataFrame(rerank_records)
rerank_summary = pd.DataFrame({
    "Metric": ["MAP", "MRR", "Precision@5", "Precision@10", "Recall@5", "Recall@10"],
    "Score":  [rerank_df["AP"].mean(), rerank_df["RR"].mean(),
               rerank_df["P@5"].mean(), rerank_df["P@10"].mean(),
               rerank_df["R@5"].mean(), rerank_df["R@10"].mean()]
})
rerank_summary


## 10) Final comparison (Dense vs BM25 vs Hybrid vs Reranked)

In [ ]:

final_comparison = pd.DataFrame({
    "Metric": ["MAP", "MRR", "Precision@5", "Precision@10", "Recall@5", "Recall@10"],
    "Dense":  [dense_df["AP"].mean(), dense_df["RR"].mean(),
               dense_df["P@5"].mean(), dense_df["P@10"].mean(),
               dense_df["R@5"].mean(), dense_df["R@10"].mean()],
    "BM25":   [bm25_df["AP"].mean(), bm25_df["RR"].mean(),
               bm25_df["P@5"].mean(), bm25_df["P@10"].mean(),
               bm25_df["R@5"].mean(), bm25_df["R@10"].mean()],
    "Hybrid": [hybrid_df["AP"].mean(), hybrid_df["RR"].mean(),
               hybrid_df["P@5"].mean(), hybrid_df["P@10"].mean(),
               hybrid_df["R@5"].mean(), hybrid_df["R@10"].mean()],
    "Reranked": [rerank_df["AP"].mean(), rerank_df["RR"].mean(),
                 rerank_df["P@5"].mean(), rerank_df["P@10"].mean(),
                 rerank_df["R@5"].mean(), rerank_df["R@10"].mean()],
})
final_comparison


## 11) (Optional) End‑to‑end generation using a small model

In [ ]:

STRATEGY = "hybrid"  # "dense"|"bm25"|"hybrid"|"reranked"
K_CONTEXT = 5
MAX_CHARS = 1800

gen = pipeline("text2text-generation", model=GEN_MODEL_NAME)

def retrieve_ids_for_strategy(i):
    q_text = query_df.iloc[i]["question"]
    if STRATEGY == "bm25":
        ranked, _ = bm25_ranked_docs_for_query(q_text)
    elif STRATEGY == "hybrid":
        ranked, _ = hybrid_ranked_docs_for_query(q_text, query_embeddings[i], doc_embeddings, alpha=0.5)
    elif STRATEGY == "reranked":
        base_ranked, _ = hybrid_ranked_docs_for_query(q_text, query_embeddings[i], doc_embeddings, alpha=0.5)
        ranked = build_full_reranked_list(q_text, base_ranked, top_n=50)
    else:
        ranked, _ = dense_ranked_docs_for_query(query_embeddings[i], doc_embeddings)
    return ranked

def build_context_text(doc_ids, k=5, max_chars=1800):
    texts = []
    for d in doc_ids[:k]:
        texts.append(doc_df.loc[d, "text"][: max_chars // k])
    return ("\n\n".join(texts))[:max_chars]

i = 0  # try different queries
q_text = query_df.iloc[i]["question"]
ranked_ids = retrieve_ids_for_strategy(i)
context_block = build_context_text(ranked_ids, k=K_CONTEXT, max_chars=MAX_CHARS)

prompt = f'''Answer the question using ONLY the context. If the answer is not in the context, say so.

Question: {q_text}

Context:
{context_block}

Answer:'''

out = gen(prompt, max_new_tokens=128, do_sample=False)
print("Q:", q_text)
print("A:", out[0]["generated_text"].strip())



## 12) Notes & next steps
- Tune **alpha** in hybrid fusion and observe metric shifts.
- Increase **TOP_N** for reranking; watch when gains saturate.
- Add **faithfulness** checks (string overlap, NLI, or LLM-as-judge).
- Replace brute-force with **FAISS** for scale.
- Try other datasets (mind licensing): **MS MARCO**, **Natural Questions**.
- Chunk contexts to create **multi-relevant** scenarios and compare MAP vs MRR behaviors.
